In [69]:
!pip install -q -U kaggle_environments

In [70]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate
import math

Опишем поведение агента, всегда играющего "камень" - это значение 0

In [71]:
%%writefile rock_agent.py

#Example of the simple agent
#0 - rock
#1 - paper
#2 - scissors
def your_agent(observation, configuration):
    return 0

Overwriting rock_agent.py


Попробуем теперь использовать информацию о прошлых действиях противника. Опишем агента, который производит то же самое действие, что и оппонент на прошлом ходу

In [72]:
%%writefile copy_opponent.py

#Example
def copy_opponent(observation, configuration):
    #in case we have information about opponent last move
    if observation.step > 0:
        return observation.lastOpponentAction
    #initial step
    else:
        return random.randrange(0, configuration.signs)

Overwriting copy_opponent.py


Воспользуемся функцией evaluate из библиотеки kaggle_environments с помощью которой запустим наших агентов и проведем эксперимент на заданном количестве игр

In [73]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "copy_opponent.py"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes
)

[[1, None]]

In [74]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "paper"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes
)

[[-99.0, 99.0]]

Из среды RPS Kaggle возьмем определение функции, вычисляющую победу агента (правый или левый)

In [114]:
def get_score(left_move, right_move):
    delta = (
        right_move - left_move
        if (left_move + right_move) % 2 == 0
        else left_move - right_move
    )
    return 0 if delta == 0 else math.copysign(1, delta)

1) Добавляем агента, который использует только камень

In [115]:
def rock_only(observation, configuration):
    # Агент, который использует только камень
    return 0

2) Добавляем агента, который использует только бумагу

In [116]:
def paper_only(observation, configuration):
    # Агент, который который использует только бумагу
    return 1

3) Добавляем агента, который использует только ножницы

In [117]:
def scissors_only(observation, configuration):
    # Агент, который использует только ножницы
    return 2

4) Добавляем агента, чья стратегия - копировать предыдущий ход оппонента

In [118]:
def copy_opponent(observation, configuration):
    # Агент, который копирует последний ход противника.
    if observation.step > 0:
        return observation.lastOpponentAction
    else:
        return random.randrange(0, configuration.signs)

5) Добавляем агента, чья стратегия - выбирать случайно

In [119]:
def random_only(observation, configuration):
    # Агент, который выбирает ход случайно (камень/бумага/ножницы)
    return random.choice([0, 1, 2])

6) Добавляем агента, чья стратегия - выбрать выигрышное действие из предыдущего хода

In [120]:
def react_prev(observation, configuration):
    if observation.step == 0:
        # возвращаем рандомное значение из доступных вариантов
        return random.randrange(0, configuration.signs)
    # берем предыдущий ход соперника и ходим, чтобы победить его
    return (observation.lastOpponentAction + 1) % configuration.signs

7) Агент, который выбирает (камень/бумага/ножницы) по кругу (исходное положение определяется случайным образом)

In [121]:
def cycle_order(observation, configuration):
    # Агент, который выбирает ход по кругу (камень/бумага/ножницы)
    if observation.step == 0:
        # если это первый ход, выбираем случайное
        return random.randrange(0, configuration.signs)
    # иначе берем остаток от деления номера текущего эпизода на количество возможных вариантов
    return observation.step % configuration.signs

In [122]:
def counter_react_prev(observation, configuration):
    global last_counter_action
    if observation.step == 0:
        last_counter_action = random.randrange(0, configuration.signs)
    elif get_score(last_counter_action, observation.lastOpponentAction) == 1:
        last_counter_action = (last_counter_action + 2) % configuration.signs
    else:
        last_counter_action = (observation.lastOpponentAction + 1) % configuration.signs

    return last_counter_action

8) Добавим агента, который побеждает циклического агента

In [123]:
def counter_cycle_order(observation, configuration):
    # Агент, который выбирает ход, который бьет циклический ход противника.
    if observation.step == 0:
        return random.randrange(0, configuration.signs)
    else:
        return (observation.lastOpponentAction + 2) % configuration.signs

9) Добавим агента, подсчитывающего наиболее частый ход оппонента, и реагирующего соответствующим образом

In [124]:
def statistical(observation, configuration):
    # Агент, который подсчитывает наиболее частый ход оппонента
    # переменная для хранения статистики
    global action_histogram
    if observation.step == 0:
        action_histogram = {}
        return
    action = observation.lastOpponentAction
    if action not in action_histogram:
        action_histogram[action] = 0
    action_histogram[action] += 1
    mode_action = None
    mode_action_count = None
    for k, v in action_histogram.items():
        if mode_action_count is None or v > mode_action_count:
            mode_action = k
            mode_action_count = v
            continue

    return (mode_action + 1) % configuration.signs

10) Добавим агента, котоырй выбирает ход, который бьет наиболее часто используемый противником ход

In [125]:
def counter_stats(observation, configuration):
    # Агент, который выбирает ход, который бьет наиболее часто используемый противником ход
    if observation.get('step', 0) > 0:
        counts = [0, 0, 0]
        for action in observation.get('history', []):
            counts[action] += 1
        return (counts.index(max(counts)) + 1) % 3
    else:
        return random.choice([0, 1, 2])

11) Адаптивный агент, который пытается предсказать следующий ход противника

In [126]:
def adaptive(observation, configuration):
    # Адаптивный агент, который пытается предсказать следующий ход противника
    # Ход, который бьет предсказанный ход противника, или случайный ход, если история недостаточно длинная
    if observation.step > 1:
        if observation.get('lastOpponentAction', 0) == observation.get('history', [0, 0])[-2]:
            return (observation.get('lastOpponentAction', 0) + 1) % 3
        else:
            return random.randrange(0, configuration.signs)
    else:
        return random.randrange(0, configuration.signs)

12) Добавим агента, стратегия которого - комбинация из стратегий 2х других агентов:
В первых 1/3 случаев - ходит как react_prev (выбрать выигрышное действие из предыдущего хода)
В дальнейшем - как statictical (выбирает ход, побеждающий наиболее частое действие соперника)

In [140]:
def one_third(observation, configuration):
    # хранение послднего хода (т.к. переменная last_step занята, используем индекс 2)
    global last_step2
    # поле для хранения  количества повторов
    global repeats
    # переменная для хранения статистики
    global action_histogram2

    if observation.step == 0:
        # если ход первый - сбрасываем статистику
        last_step2 = 0
        repeats = 0
        action_histogram2 = {}

        # возвращаем случайное значение
        return random.randrange(0, configuration.signs)
    # сохраняем в переменную последний ход соперника
    action = observation.lastOpponentAction
    # если количество повторов меньше 1/3 от запланированного количества эпизодов
    if repeats < (configuration.get('episodeSteps') / 3):
        # увеличиваем счётчик повторов на 1
        repeats += 1
        # ходим с учётом предыдущего хода соперника
        return (action + 1) % configuration.signs
    # иначе (после прохождения 1/3 эпизодов)
    else:
        # если такого ключа еще нет в справочнике
        if action not in action_histogram2:
            # добавляем в справочник со значением 0
            action_histogram2[action] = 0
        # увеличиваем счётчик для такого хода соперника
        action_histogram2[action] += 1
        # определяем статистически самый частый ход соперника
        # инициализируем переменную для хранения хода с максимальным счетчиком
        mode_action = None
        # инициализируем переменную для хранения максимального значения счетчика
        mode_action_count = None
        # для каждой пары ключ-значение в справочнике статистики
        for k, v in action_histogram2.items():
            # если значение в справочнике для этого ключа не установлено
            # или значение итератора больше значения в справочнике
            if mode_action_count is None or v > mode_action_count:
                # сохраняем текущий ключ в переменную максимума
                mode_action = k
                # сохраняем текущее значение в переменную максимума
                mode_action_count = v
                # переходим к следующей итераци
                continue
        # ходим так, чтобы победить самый частый ход соперника
        return (mode_action + 1) % configuration.signs

Итоговый список агентов

In [141]:
'''
agents = {
    'rock': rock_only,
    'paper': paper_only,
    'scissors': scissors_only,
    'copy': copy_opponent,
    'random': random_only,
    'react': react_prev,
    'cycle': cycle_order,
    'c_react': counter_react_prev,
    'c_cycle': counter_cycle_order,
    'stats': statistical,
    'c_stats': counter_stats,
    'adapt': adaptive,
    '1/3': one_third
}
'''

"\nagents = {\n    'rock': rock_only,\n    'paper': paper_only,\n    'scissors': scissors_only,\n    'copy': copy_opponent,\n    'random': random_only,\n    'react': react_prev,\n    'cycle': cycle_order,\n    'c_react': counter_react_prev,\n    'c_cycle': counter_cycle_order,\n    'stats': statistical,\n    'c_stats': counter_stats,\n    'adapt': adaptive,\n    '1/3': one_third\n}\n"

In [142]:
agents = [
    rock_only,
    paper_only,
    scissors_only,
    copy_opponent,
    random_only,
    react_prev,
    cycle_order,
    counter_react_prev,
    counter_cycle_order,
    statistical,
    counter_stats,
    adaptive,
    one_third
]

In [143]:
from kaggle_environments import make, evaluate
from collections import defaultdict

# Запуск турнира
results = []
for i in range(len(agents)):
    for j in range(i + 1, len(agents)):
        agent1 = agents[i]
        agent2 = agents[j]
        match_result = evaluate(
            "rps",  # environment to use
            [agent1, agent2],  # agents to evaluate
            configuration={"episodeSteps": 100}  # number of episodes
        )
        results.append((agent1.__name__, agent2.__name__, match_result))

# Вывод результатов
for result in results:
    print(f"Agent 1: {result[0]}, Agent 2: {result[1]}, Result: {result[2]}")

# Подсчет результатов
agent_scores = defaultdict(lambda: {"wins": 0, "losses": 0, "draws": 0})

for result in results:
    agent1_name = result[0]
    agent2_name = result[1]
    match_result = result[2][0]

    # Проверка на корректность результатов
    if match_result[0] is None or match_result[1] is None:
        print(f"Error in match between {agent1_name} and {agent2_name}: One of the agents returned None.")
        continue

    if match_result[0] > match_result[1]:
        agent_scores[agent1_name]["wins"] += 1
        agent_scores[agent2_name]["losses"] += 1
    elif match_result[0] < match_result[1]:
        agent_scores[agent1_name]["losses"] += 1
        agent_scores[agent2_name]["wins"] += 1
    else:
        agent_scores[agent1_name]["draws"] += 1
        agent_scores[agent2_name]["draws"] += 1

# Вывод результатов
for agent, scores in agent_scores.items():
    print(f"Agent: {agent}, Wins: {scores['wins']}, Losses: {scores['losses']}, Draws: {scores['draws']}")

Agent 1: rock_only, Agent 2: paper_only, Result: [[-99.0, 99.0]]
Agent 1: rock_only, Agent 2: scissors_only, Result: [[99.0, -99.0]]
Agent 1: rock_only, Agent 2: copy_opponent, Result: [[0, 0]]
Agent 1: rock_only, Agent 2: random_only, Result: [[0, 0]]
Agent 1: rock_only, Agent 2: react_prev, Result: [[-98.0, 98.0]]
Agent 1: rock_only, Agent 2: cycle_order, Result: [[0, 0]]
Agent 1: rock_only, Agent 2: counter_react_prev, Result: [[-49.0, 49.0]]
Agent 1: rock_only, Agent 2: counter_cycle_order, Result: [[98.0, -98.0]]
Agent 1: rock_only, Agent 2: statistical, Result: [[-98.0, 98.0]]
Agent 1: rock_only, Agent 2: counter_stats, Result: [[-97.0, 97.0]]
Agent 1: rock_only, Agent 2: adaptive, Result: [[-96.0, 96.0]]
Agent 1: rock_only, Agent 2: one_third, Result: [[-99.0, 99.0]]
Agent 1: paper_only, Agent 2: scissors_only, Result: [[-99.0, 99.0]]
Agent 1: paper_only, Agent 2: copy_opponent, Result: [[0, 0]]
Agent 1: paper_only, Agent 2: random_only, Result: [[0, 0]]
Agent 1: paper_only, Age

Лучшие агенты

Agent: react_prev, Wins: 8, Losses: 0, Draws: 4

Agent: statistical, Wins: 6, Losses: 1, Draws: 5

Agent: one_third, Wins: 6, Losses: 2, Draws: 4